# 13. 희소 코딩 (Sparse Coding)

데이터를 희소한 표현으로 분해하는 방법을 학습합니다.

## 학습 목표
- 희소 표현의 개념 이해
- 딕셔너리 학습 (Dictionary Learning)
- L1 정규화와 희소성

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 희소 표현이란?

신호 x를 딕셔너리 D의 열(atom)들의 선형 조합으로 표현:
$$x \approx D\alpha$$

여기서 α는 희소(sparse)해야 합니다 (대부분 0).

In [ ]:
# 간단한 예: 1D 신호
n = 100  # 신호 길이
k = 20   # 딕셔너리 atom 수

# 기본 딕셔너리: DCT-like 기저
D = torch.zeros(n, k)
for i in range(k):
    freq = (i + 1) * np.pi / n
    D[:, i] = torch.cos(torch.arange(n).float() * freq)

# 정규화
D = D / D.norm(dim=0, keepdim=True)

# 희소 계수로 신호 생성
alpha_true = torch.zeros(k)
alpha_true[2] = 1.5
alpha_true[7] = -1.0
alpha_true[15] = 0.8

x = D @ alpha_true + torch.randn(n) * 0.1  # 노이즈 추가

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# 원본 신호
axes[0, 0].plot(x)
axes[0, 0].set_title('Signal x')
axes[0, 0].set_xlabel('Sample')

# 딕셔너리 일부
for i in range(5):
    axes[0, 1].plot(D[:, i], label=f'Atom {i}')
axes[0, 1].set_title('Dictionary Atoms (first 5)')
axes[0, 1].legend()

# True 희소 계수
axes[1, 0].stem(alpha_true)
axes[1, 0].set_title('True Sparse Coefficients α')
axes[1, 0].set_xlabel('Atom index')

# 재구성
x_recon = D @ alpha_true
axes[1, 1].plot(x, label='Original + noise')
axes[1, 1].plot(x_recon, '--', label='Reconstruction')
axes[1, 1].set_title('Reconstruction')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 2. ISTA (Iterative Shrinkage-Thresholding Algorithm)

희소 코딩 문제를 풀기 위한 알고리즘:
$$\min_\alpha \frac{1}{2}\|x - D\alpha\|_2^2 + \lambda\|\alpha\|_1$$

In [ ]:
def soft_threshold(x, threshold):
    """Soft thresholding (shrinkage) 연산"""
    return torch.sign(x) * torch.relu(torch.abs(x) - threshold)

def ista(x, D, lambda_reg=0.1, max_iters=100):
    """ISTA 알고리즘"""
    n, k = D.shape
    alpha = torch.zeros(k)
    
    # 스텝 크기
    L = torch.linalg.eigvalsh(D.T @ D).max()
    step = 1.0 / L
    
    for _ in range(max_iters):
        # Gradient step
        grad = D.T @ (D @ alpha - x)
        alpha = alpha - step * grad
        
        # Proximal (shrinkage) step
        alpha = soft_threshold(alpha, lambda_reg * step)
    
    return alpha

# ISTA 실행
alpha_recovered = ista(x, D, lambda_reg=0.1, max_iters=200)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].stem(alpha_true, linefmt='b-', markerfmt='bo', label='True')
axes[0].stem(alpha_recovered, linefmt='r--', markerfmt='rx', label='Recovered')
axes[0].set_title('Sparse Coefficients')
axes[0].legend()

x_recon = D @ alpha_recovered
axes[1].plot(x, label='Original')
axes[1].plot(x_recon, '--', label='Reconstruction')
axes[1].set_title(f'Reconstruction (MSE: {((x - x_recon)**2).mean():.4f})')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"True sparsity: {(alpha_true != 0).sum().item()}")
print(f"Recovered sparsity: {(alpha_recovered.abs() > 0.01).sum().item()}")

## 3. 이미지 패치에서의 희소 코딩

In [ ]:
# 이미지 패치 생성
patch_size = 8
n_patches = 500

# 간단한 패턴 생성
patches = []
for _ in range(n_patches):
    patch = torch.zeros(patch_size, patch_size)
    
    # 랜덤 에지 또는 그라디언트
    pattern = np.random.choice(['h_edge', 'v_edge', 'diag', 'spot'])
    
    if pattern == 'h_edge':
        pos = np.random.randint(2, patch_size-2)
        patch[:pos, :] = 1
    elif pattern == 'v_edge':
        pos = np.random.randint(2, patch_size-2)
        patch[:, :pos] = 1
    elif pattern == 'diag':
        for i in range(patch_size):
            patch[i, :i] = 1
    else:  # spot
        cx, cy = np.random.randint(1, patch_size-1, 2)
        patch[cx-1:cx+2, cy-1:cy+2] = 1
    
    patch += torch.randn_like(patch) * 0.1
    patches.append(patch.flatten())

X = torch.stack(patches)  # (n_patches, patch_size^2)
print(f"Data shape: {X.shape}")

# 샘플 패치 시각화
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(X[i].reshape(patch_size, patch_size), cmap='gray')
    ax.axis('off')
plt.suptitle('Sample Patches')
plt.tight_layout()
plt.show()

In [ ]:
from mlfs.probabilistic.sparse import SparseCoding

# 희소 코딩 학습
n_atoms = 64
sc = SparseCoding(n_atoms=n_atoms, lambda_reg=0.1, max_iters=50)
sc.fit(X)

print(f"Dictionary shape: {sc.dictionary.shape}")

In [ ]:
# 학습된 딕셔너리 시각화
D = sc.dictionary

fig, axes = plt.subplots(8, 8, figsize=(10, 10))
for i, ax in enumerate(axes.flatten()):
    if i < n_atoms:
        atom = D[:, i].reshape(patch_size, patch_size)
        ax.imshow(atom, cmap='gray')
    ax.axis('off')

plt.suptitle('Learned Dictionary Atoms', fontsize=14)
plt.tight_layout()
plt.show()

## 4. 재구성 품질

In [ ]:
# 테스트 패치 재구성
test_idx = [0, 10, 50, 100, 200]

fig, axes = plt.subplots(len(test_idx), 3, figsize=(8, 12))

for row, idx in enumerate(test_idx):
    x = X[idx]
    
    # 희소 코드 계산
    alpha = sc.encode(x.unsqueeze(0)).squeeze()
    
    # 재구성
    x_recon = D @ alpha
    
    # 원본
    axes[row, 0].imshow(x.reshape(patch_size, patch_size), cmap='gray')
    axes[row, 0].set_title('Original' if row == 0 else '')
    axes[row, 0].axis('off')
    
    # 재구성
    axes[row, 1].imshow(x_recon.reshape(patch_size, patch_size), cmap='gray')
    axes[row, 1].set_title('Reconstructed' if row == 0 else '')
    axes[row, 1].axis('off')
    
    # 희소 코드
    axes[row, 2].stem(alpha.abs())
    axes[row, 2].set_title(f'Sparse Code ({(alpha.abs() > 0.01).sum()} active)' if row == 0 else f'{(alpha.abs() > 0.01).sum()} active')
    if row == len(test_idx) - 1:
        axes[row, 2].set_xlabel('Atom index')

plt.tight_layout()
plt.show()

## 5. 희소성과 재구성 품질 트레이드오프

In [ ]:
# 다양한 lambda 값으로 실험
lambdas = [0.01, 0.05, 0.1, 0.2, 0.5]

test_x = X[0]

fig, axes = plt.subplots(2, len(lambdas), figsize=(15, 6))

for col, lam in enumerate(lambdas):
    # 희소 코드 계산 (ISTA 직접 사용)
    alpha = ista(test_x, D, lambda_reg=lam, max_iters=200)
    x_recon = D @ alpha
    
    mse = ((test_x - x_recon)**2).mean().item()
    sparsity = (alpha.abs() > 0.01).sum().item()
    
    # 재구성
    axes[0, col].imshow(x_recon.reshape(patch_size, patch_size), cmap='gray')
    axes[0, col].set_title(f'λ={lam}\nMSE: {mse:.4f}')
    axes[0, col].axis('off')
    
    # 희소 코드
    axes[1, col].stem(alpha.abs())
    axes[1, col].set_title(f'{sparsity} active atoms')

axes[0, 0].set_ylabel('Reconstruction', fontsize=12)
axes[1, 0].set_ylabel('Sparse Code', fontsize=12)

plt.suptitle('Effect of Regularization Strength', fontsize=14)
plt.tight_layout()
plt.show()

## 요약

1. **희소 코딩**: 데이터를 소수의 기저 요소로 표현
2. **L1 정규화**: 희소성 유도 (많은 계수가 0)
3. **ISTA**: 근위 경사하강법으로 최적화
4. **딕셔너리 학습**: 데이터에서 최적의 기저 학습
5. **응용**: 이미지 압축, 노이즈 제거, 특징 학습

## 축하합니다!

이것으로 ml-from-scratch 튜토리얼을 모두 완료했습니다. 학습한 내용:
- 신경망: Perceptron, MLP, CNN
- 클러스터링: K-Means, Spectral Clustering
- 차원 축소: PCA, LLE, Isomap
- 앙상블: AdaBoost
- 확률 모델: EM/GMM, Sparse Coding
- 컴퓨터 비전: 얼굴 검출 개념